In [237]:
import numpy as np
import pandas as pd
#import scipy
from datetime import date, datetime

In [238]:

users = pd.read_csv(r"D:\dataset\users_cleaned.csv")
anime = pd.read_csv(r"D:\dataset\anime_cleaned.csv")
ratings = pd.read_csv(r"D:\dataset\animelists_cleaned.csv", nrows = 1000000)
af = pd.DataFrame(anime)
rf = pd.DataFrame(ratings)


Generating additional user fetures for data embedding 

In [289]:
#GENERATE USER VECTOR

af = af.loc[af["type"].isin(["TV","Movie"]) & (af["rating"]!="Rx - Hentai")] #only include TV shows and Movies and remove nsfw shows
uf = users.loc[:, ["user_id", "user_completed", "user_days_spent_watching", "gender", "birth_date", "stats_episodes"]]
uf["birth_date"] = pd.to_datetime(uf["birth_date"])

#Only include users who have completed at least 5 anime
uf = uf[uf["user_completed"] >= 5]

#CREATE GENERATION FEATURE
#Convert birth years to generations
def get_birth_year(date):
    birth_year = date.strftime("%Y")
    return int(birth_year)

uf["birth_date"] = uf["birth_date"].apply(get_birth_year)

uf = uf.rename(columns={"birth_date":"generation"})

#Assign generations based on birth year
generations = [
    (uf["generation"] >= 2013) & (uf["generation"] <= 2025),
    (uf["generation"] >= 1995) & (uf["generation"] <= 2012),
    (uf["generation"] >= 1980) & (uf["generation"] <= 1994),
    (uf["generation"] >= 1965) & (uf["generation"] <= 1979)
]

gen_values = [
    "Gen Alpha",
    "Gen Z",
    "Millenials",
    "Gen X"
]
uf["generation"] = np.select(generations, gen_values, default=uf["generation"])

#CREATE EXPERIENCE FEATURE
#Cast days_watched to integers so that users are not missed in conditions below
uf["user_days_spent_watching"] = uf["user_days_spent_watching"].astype("int")
#Assign experience levels based on days watched
user_exp = [
    (uf["user_days_spent_watching"] >= 0) & (uf["user_days_spent_watching"] <= 5),
    (uf["user_days_spent_watching"] >= 6) & (uf["user_days_spent_watching"] <= 50),
    (uf["user_days_spent_watching"] >= 51) & (uf["user_days_spent_watching"] <= 99),
    (uf["user_days_spent_watching"] >= 100)
]

exp_values = [
    "Newbie",
    "Regular",
    "Active",
    "Veteran"
]

uf["user_days_spent_watching"] = np.select(user_exp, exp_values, default=uf["user_days_spent_watching"])
uf = uf.rename(columns={"user_days_spent_watching":"experience"})
uf


#CREATE FAVOURITE GENRE FEATURE

#Only include users who have rated more than 10 anime
num_user_ratings = rf.groupby("username").size() 
limit = num_user_ratings[num_user_ratings >= 10].index
usernames = rf["username"].isin(limit)
rf = rf.loc[usernames]

#Merge the ratings and anime dataframes
merge_rf = pd.merge(rf, af, on="anime_id")
merge_rf = merge_rf.loc[:, ["username", "anime_id","title","genre","type", "my_score","my_watched_episodes", "episodes"]]

#Remove all anime ratings where the user has watched less than 10 episodes unless it is a movie
movies = merge_rf[merge_rf["type"]=="Movie"]
valid_tv_shows = merge_rf[(merge_rf["type"]=="TV") & (merge_rf["my_watched_episodes"]>=10)]
merge_rf = pd.concat([movies, valid_tv_shows])

#Sort by highest rated anime by each user
merge_rf.set_index("username")
merge_rf = merge_rf.groupby("username").apply(lambda i:i.sort_values(by="my_score", ascending=False))
merge_rf.reset_index(drop=True, inplace=True)

#Only include top 10 highest rated anime per user
merge_rf = merge_rf.groupby("username").head(10)
                                                             
#Example user/ must make work for all users
user = "Damonashu"
user_ratings = merge_rf.loc[merge_rf["username"]==user]

#Creating a list all genres in the top 10 for each user (including duplicates)
fav_list = [] #list of most popular genres for each user
user_genres = []

#Concatenate genres for each anime into one list
for i in user_ratings["genre"]:
    x = i.split()
    user_genres = user_genres + x

final_result = []
for i,v in enumerate(user_genres):
    if v!="nan":
        new = v.replace(" ", "")
        new = v.replace(",", "")
        user_genres[i]=new
        

#Find top 3 favourite most frequent genres in the top 10
for i in range(0, 3):
    most_popular = max(set(user_genres), key=user_genres.count)
    fav_list.append(most_popular)
    #Remove genre from the list once added to favourites
    for i in range(0, user_genres.count(most_popular)):
        user_genres.remove(most_popular)


#test

#for i in merge_rf.groupby("username")["genre"]:
#    user_ratings = merge_rf.loc[merge_rf["username"]==i[0]]
  
    #print(user_ratings["username"])
#print(fav_list)
#user_ratings

merge_rf

#put this list in merge with user df
#change num_episodes to anime length feature
#user_id  |  fav_genre | preferred anime length | new-gen/classic
#account for number of episodes watched
#get most popular genres 
#create the feature
#limitation is that existing users' favourite genres are generated by implication compared to 
# new user who is asked directly


#new features
#Favourite genre(s)
#Perferred anime length

#4. see if any shows appear in multiple people's top 10 lists
#Train set and test set
#Then use lightfm in a new version

#take propensity to rewatch: rewatched/watched (rw)
#take propensity to drop: dropped/watched (dr)
#cosine similarity of all users and movies takes a lot of resources

,username,anime_id,title,genre,type,my_score,my_watched_episodes,episodes
0,-Ackerman,32,Neon Genesis Evangelion: The End of Evangelion,"Sci-Fi, Dementia, Psychological, Drama, Mecha",Movie,10,1,1
1,-Ackerman,10165,Nichijou,"Slice of Life, Comedy, School",TV,10,26,26
2,-Ackerman,5081,Bakemonogatari,"Romance, Supernatural, Mystery, Vampire",TV,10,15,15
3,-Ackerman,5114,Fullmetal Alchemist: Brotherhood,"Action, Military, Adventure, Comedy, Drama, Ma...",TV,10,64,64
4,-Ackerman,6702,Fairy Tail,"Action, Adventure, Comedy, Magic, Fantasy, Sho...",TV,10,175,175
...,...,...,...,...,...,...,...,...
550613,zzs,4898,Kuroshitsuji,"Action, Comedy, Demons, Fantasy, Historical, S...",TV,8,24,24
550614,zzs,60,Chrno Crusade,"Action, Historical, Demons, Supernatural, Romance",TV,8,24,24
550615,zzs,167,Scrapped Princess,"Adventure, Comedy, Drama, Fantasy, Mecha, Sci-...",TV,8,24,24
550616,zzs,1974,Glass no Kamen (2005),"Drama, Shoujo",TV,8,51,51


In [284]:
#Building a dictionary of all genres in a dataframe
def build_genre_list(dataframe):

    def split(genres):
        return str(genres).split(",")

    split_genres = dataframe["genre"].apply(split)
    #Updating the set to get the individual genre names in the set instead of the original list of genres
    genre_set = set()
    for i in split_genres: 
        genre_set.update(i)
    result = list(genre_set)

    #Removing blank spaces
    final_result = []
    for i in result:
        if i!="nan":
            new = i.replace(" ", "")
            final_result.append(new)

    #Removing duplicates now that spaces have been removed
    genre_list = list(set(final_result))
    print(genre_list)

user_genres = build_genre_list(af)



#anime vector
#filter anime that only have a certain number of ratings
#feature: new-gen/classic/medium
#consider the length of the anime. Ranges: short, medium, long
#consider production studio
#label encoding for normal fields
#one-hot encoding (binary) for genres

#Go through anime_cleaned file
#Merge this file with the users file
#Find all the users that rated each anime
#Find what those users rated those anime and which groups those users belong to
# So you know users from group X liked anime Y "this" much
#Genre: Make each genre its own binary column

#consider number of ratings anime has
#Exclude "hentai genre"
#remember to split the data
#all "currently running" anime show up as having 0 episodes
#filter type to TV only
#give user ability to choose long or short animes


#af = pd.DataFrame(anime)
#af = af.loc[pd.isna(af["title_english"]) == False,:]
#af



['School', 'Parody', 'Police', 'Vampire', 'Seinen', 'ShoujoAi', 'MartialArts', 'Dementia', 'Shounen', 'Harem', 'Mecha', 'Historical', 'ShounenAi', 'Kids', 'Ecchi', 'Josei', 'Space', 'Sports', 'Action', 'Comedy', 'Demons', 'Romance', 'Samurai', 'Fantasy', 'Supernatural', 'Game', 'Horror', 'Mystery', 'Sci-Fi', 'Military', 'Shoujo', 'SuperPower', 'Cars', 'Magic', 'Psychological', 'Music', 'SliceofLife', 'Thriller', 'Drama', 'Adventure']
